## Доверительные интервалы

### Practice Quiz: Доверительные интервалы для среднего

#### 2.

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

**Будьте осторожны при использовании метода std()!** Дело в том, что у объекта numpy он по умолчанию вычисляется как $\sqrt{\frac1{n}\sum\limits_{i=1}^n\left(X_i-\bar{X}\right)^2}$ а у объекта pandas — как $\sqrt{\frac1{n-1}\sum\limits_{i=1}^n\left(X_i-\bar{X}\right)^2}$

Нас интересует только второй вариант, несмещённая оценка стандартного отклонения.

Чтобы не думать всё время о том, правильно ли вычисляется в вашем случае std(), можно всегда использовать std(ddof=1) (ddof — difference in degrees of freedom), тогда нормировка всегда будет на n-1.

In [2]:
import math
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

data = pd.read_csv('water.txt', delimiter='\t')
data.head(5)

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [3]:
def get_interval(column):
    mean, std = column.mean(), column.std(ddof=1)
    return _tconfint_generic(mean, std/np.sqrt(len(column)), len(column) - 1, 0.05, 'two-sided')

In [4]:
round(get_interval(data['mortality'])[0], ndigits=4)

1476.0833

#### 3.

На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [5]:
round(get_interval(data[data.location == 'South']['mortality'])[1], ndigits=4)

1433.4636

#### 4.

На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод?

In [6]:
get_interval(data[data.location == 'North']['mortality'])

(1586.5605251961385, 1680.6394748038613)

#### 5.
Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?

In [7]:
get_interval(data[data.location == 'North']['hardness']), get_interval(data[data.location == 'South']['hardness'])

((21.422487285724259, 39.377512714275738),
 (53.467198692036106, 86.071262846425441))

#### 6.

Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией $\sigma^2$: 

$\bar{X}_n\pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}$

При $\sigma=1$ какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью $\pm0.1$?

In [8]:
# TODO

### Practice Quiz: Доверительные интервалы для долей

In [9]:
# TODO

### Quiz: Доверительные интервалы

#### 1.

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X\sim N\left(\mu,\sigma^2\right)$ лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы $c$? Округлите ответ до четырёх знаков после десятичной точки.

In [10]:
import scipy.stats
round(scipy.stats.norm.interval(0.997, loc=0, scale=1)[1], ndigits=4)

2.9676999999999998

#### 5.

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [11]:
aspirin = 11037
placebo = 11034

aspirin_heart_attack = 104
placebo_heart_attack = 189

round(189/11034 - 104/11037, ndigits=4)

0.0077

#### 6.

Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [12]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

sample1 = np.array([1] * placebo_heart_attack + [0] * (placebo - placebo_heart_attack))
sample2 = np.array([1] * aspirin_heart_attack + [0] * (aspirin - aspirin_heart_attack))
proportions_confint_diff_ind(sample1, sample2)

(0.0046877506750494392, 0.010724297276960124)

#### 7.

Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин $X\sim Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $\hat{p}$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174$

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [13]:
round((189/(11034-189))/(104/(11037-104)), ndigits=4)

1.8321

#### 8.

Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

* составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
* установите random seed=0;
* сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [14]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

def get_odds(sample):
    s = sample.sum()
    return s/(len(sample) - s)

In [ ]:
np.random.seed(0)

a = get_bootstrap_samples(sample1, 1000)
b = get_bootstrap_samples(sample2, 1000)

odds_a = list(map(get_odds, a))
odds_b = list(map(get_odds, b))

odds = np.array(odds_a)/np.array(odds_b)

stat_intervals(odds, 0.05)

### Quiz: Практика проверки гипотез

#### 1.

По данным опроса, 75% работников ресторанов утверждают, что испытывают на работе существенный стресс, оказывающий негативное влияние на их личную жизнь. Крупная ресторанная сеть опрашивает 100 своих работников, чтобы выяснить, отличается ли уровень стресса работников в их ресторанах от среднего. 67 из 100 работников отметили высокий уровень стресса.

Посчитайте достигаемый уровень значимости, округлите ответ до четырёх знаков после десятичной точки.

In [33]:
scipy.stats.binom_test(67, 100, 0.75, alternative='two-sided')

0.082222588913866079

#### 3.

The Wage Tract — заповедник в округе Тома, Джорджия, США, деревья в котором не затронуты деятельностью человека со времён первых поселенцев. Для участка заповедника размером 200х200 м имеется информация о координатах сосен (sn — координата в направлении север-юг, we — в направлении запад-восток, обе от 0 до 200).

Проверим, можно ли пространственное распределение сосен считать равномерным, или они растут кластерами.

Загрузите данные, поделите участок на 5х5 одинаковых квадратов размера 40x40 м, посчитайте количество сосен в каждом квадрате (чтобы получить такой же результат, как у нас, используйте функцию scipy.stats.binned_statistic_2d).

Если сосны действительно растут равномерно, какое среднее ожидаемое количество сосен в каждом квадрате? В правильном ответе два знака после десятичной точки.

In [16]:
pines = pd.read_csv('pines.txt', delimiter='\t')
pines.head(5)

,sn,we
0,200.0,8.8
1,199.3,10.0
2,193.6,22.4
3,167.7,35.6
4,183.9,45.4


In [37]:
stat = scipy.stats.binned_statistic_2d(pines['we'], pines['sn'], None, 'count', bins=[5, 5])
stat.statistic.mean()

23.359999999999999

#### 4.

Чтобы сравнить распределение сосен с равномерным, посчитайте значение статистики хи-квадрат для полученных 5х5 квадратов. Округлите ответ до двух знаков после десятичной точки.

In [38]:
chi = scipy.stats.chisquare(stat.statistic.reshape(25))
chi.statistic

150.58904109589042

#### 5.

Насколько велико это значение? Если нулевая гипотеза справедлива, с какой вероятностью его можно было получить случайно?

Нулевое распределение статистики — хи-квадрат с $25−1=24$ степенями свободы (поскольку у равномерного распределения, с которым мы сравниваем данные, нет ни одного оцениваемого по выборке параметра, число степеней свободы $K−1$, где $K$ — количество интервалов).

Посчитайте достигаемый уровень значимости.

Если вы используете функцию scipy.stats.chi2.cdf, в качестве значения параметра df нужно взять 24 (это число степеней свободы); если функцию scipy.stats.chisquare — параметр ddof нужно брать равным 0 (это как раз количество параметров теоретического распределения, оцениваемых по выборке).

Отвергается ли гипотеза равномерности на уровне значимости 0.05?

In [39]:
chi.pvalue

2.5746697749672791e-20